In [1]:
%pwd

import os
os.getcwd()


'c:\\Users\\riswa\\Desktop\\AI\\LLMS-development\\Medical Assistance'

In [2]:
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from pathlib import Path
from typing import List, Union

def load_pdf(file_location: Union[str, Path]) -> List:
    """
    Load PDF document(s) from the given file location.
    
    Args:
        file_location (Union[str, Path]): Path to PDF file or directory containing PDFs
        
    Returns:
        List: List of loaded PDF pages/documents
        
    Raises:
        FileNotFoundError: If the file or directory doesn't exist
        ValueError: If the path is invalid or no PDFs are found
    """
    try:
        # Convert string path to Path object for better handling
        file_path = Path(file_location)
        
        # Check if path exists
        if not file_path.exists():
            raise FileNotFoundError(f"The path {file_path} does not exist")
            
        # Handle directory of PDFs
        if file_path.is_dir():
            loader = DirectoryLoader(
                str(file_path),
                glob="**/*.pdf",  # Load PDFs from all subdirectories
                loader_cls=PyPDFLoader
            )
            documents = loader.load()
            if not documents:
                raise ValueError(f"No PDF files found in directory {file_path}")
            return documents
            
        # Handle single PDF file
        elif file_path.is_file() and file_path.suffix.lower() == '.pdf':
            loader = PyPDFLoader(str(file_path))
            return loader.load()
            
        else:
            raise ValueError(f"The path {file_path} is not a PDF file or directory")
            
    except Exception as e:
        raise Exception(f"Error loading PDF: {str(e)}")

# Example usage:
if __name__ == "__main__":
    # Example with single PDF
    try:
        single_pdf = load_pdf("Medical_book.pdf")
        print(f"Loaded {len(single_pdf)} pages from single PDF")
    except Exception as e:
        print(f"Error with single PDF: {e}")
        
    # Example with directory of PDFs
    try:
        pdf_directory = load_pdf("Medical_book.pdf")
        print(f"Loaded {len(pdf_directory)} documents from directory")
    except Exception as e:
        print(f"Error with PDF directory: {e}")


Loaded 637 pages from single PDF
Loaded 637 documents from directory


In [3]:
def text_splitter(text):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = text_splitter.split_documents(text)
    return chunks



In [4]:

text_splitter(single_pdf)[0]
single_pdf[0]


Document(metadata={'source': 'Medical_book.pdf', 'page': 0}, page_content='')

In [11]:
text_splitter(single_pdf)[0]

Document(metadata={'source': 'Medical_book.pdf', 'page': 1}, page_content='The GALE\nENCYCLOPEDIA\nof MEDICINE\nSECOND EDITION')

In [ ]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

def download_embeddings():
    embeddings = HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

embeddings = download_embeddings()

embeddings.embed_query("What is the meaning of life?")


In [6]:
from dotenv import load_dotenv

load_dotenv()
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
GROQ_API_KEY=os.environ.get('GROQ_API_KEY')

os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["GROQ_API_KEY"] = GROQ_API_KEY


In [8]:
PINECONE_API_KEY

'pcsk_5G6ane_8XgZ4N4q55DSFs6b5UAtaFhT2VrAUao437QXswDasLrUimTvp23yjHsWyiJBeY7'

In [9]:
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore

pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "medical-chatbot"  # Changed to lowercase with hyphen

pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

In [32]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY


In [11]:
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore
import os

# Initialize Pinecone with the API key from environment variable
# PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
# pc = Pinecone(api_key=PINECONE_API_KEY)
# index_name = "medical-chatbot"  # Changed to lowercase with hyphen

# # Create the index
# pc.create_index(
#     name=index_name,
#     dimension=384,
#     metric="cosine",
#     spec=ServerlessSpec(
#         cloud="aws",
#         region="us-east-1"
#     )
# )

# Create text splitter function if not already defined
def text_splitter(documents):
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )
    return text_splitter.split_documents(documents)

# Assuming single_pdf is defined elsewhere in your code
text_chunks = text_splitter(single_pdf)

# Assuming embeddings is defined elsewhere in your code
# Create vector store from documents
docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings
)


In [12]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [13]:
retriever.invoke("What is Acne?")

[Document(id='2ca28fb0-375c-4dfe-80cd-d367a329dbb7', metadata={'page': 37.0, 'source': 'Medical_book.pdf'}, page_content='Description\nAcne vulgaris, the medical term for common acne, is\nthe most common skin disease. It affects nearly 17 million\npeople in the United States. While acne can arise at any\nage, it usually begins at puberty and worsens during ado-\nlescence. Nearly 85% of people develop acne at some time\nbetween the ages of 12-25 years. Up to 20% of women\ndevelop mild acne. It is also found in some newborns.\nThe sebaceous glands lie just beneath the skin’s sur-\nface. They produce an oil called sebum, the skin’s natural\nmoisturizer. These glands and the hair follicles within\nwhich they are found are called sebaceous follicles.\nThese follicles open onto the skin through pores. At\npuberty, increased levels of androgens (male hormones)\ncause the glands to produce too much sebum. When\nexcess sebum combines with dead, sticky skin cells, a\nhard plug, or comedo, forms 